In [ ]:
# This is necessary to recognize the modules
import os
import sys
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(root_path)

In [ ]:
from core.data_sources import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

In [ ]:
candles = await clob.get_candles_last_days(connector_name="binance", trading_pair="BTC-USDT", interval="1m", days=10)

In [ ]:
from core.features.candles.mean_reversion_channel import MeanReversionChannel, MeanReversionChannelConfig

config = MeanReversionChannelConfig(length=200)

candles.add_feature(MeanReversionChannel(config))

In [ ]:
df = candles.data

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from core.data_structures.candles import Candles
from core.features.candles.mean_reversion_channel import MeanReversionChannel, MeanReversionChannelConfig


def compute_signal(df):
    # Convert index to datetime if it's not already
    df.index = pd.to_datetime(df.index)

    # Initialize signals
    df["signal"] = 0

    # Group by date
    grouped = df.groupby(df.index.date)

    for date, group in grouped:
        # Compute open prices for 8, 9, 10, 11 hours for this specific day
        open_prices = group.between_time("08:00", "11:59")["open"].resample("1H").first()

        if not open_prices.empty:
            min_open = open_prices.min()
            max_open = open_prices.max()

            # Create time mask for trading hours (12:00 to 15:00)
            time_mask = (group.index.time >= pd.Timestamp("12:00").time()) & (group.index.time <= pd.Timestamp("15:00").time())

            # Apply conditions
            buy_condition = (group["low"] <= group["loband2"]) & (group["close"] < min_open)
            sell_condition = (group["high"] >= group["upband2"]) & (group["close"] > max_open)

            # Combine conditions
            combined_mask = time_mask & (buy_condition | sell_condition)

            # Set signals
            df.loc[group.index[combined_mask & buy_condition], "signal"] = 1  # Buy signal
            df.loc[group.index[combined_mask & sell_condition], "signal"] = -1  # Sell signal

    return df


def plot_mean_reversion_channel(candles: Candles, candles_with_mrc: pd.DataFrame):
    # Create the figure with two rows
    fig = make_subplots(
        rows=2,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.7, 0.3],
        specs=[[{"secondary_y": False}], [{"secondary_y": True}]],
    )

    # Add candlestick chart to the first row
    candlestick = candles.candles_trace()
    fig.add_trace(candlestick, row=1, col=1)

    # Add Mean Line to the first row
    fig.add_trace(
        go.Scatter(
            x=candles_with_mrc.index,
            y=candles_with_mrc["meanline"],
            mode="lines",
            name="Mean Line",
            line=dict(color="#FFCD00", width=2),
        ),
        row=1,
        col=1,
    )

    # Add Upper and Lower Bands
    for band, color in [("upband1", "green"), ("loband1", "green"), ("upband2", "red"), ("loband2", "red")]:
        fig.add_trace(
            go.Scatter(
                x=candles_with_mrc.index,
                y=candles_with_mrc[band],
                mode="lines",
                name=f"{band.capitalize()}",
                line=dict(color=color, width=1),
                opacity=0.5,
            ),
            row=1,
            col=1,
        )
    buy_signals = candles_with_mrc[candles_with_mrc["signal"] == 1]
    fig.add_trace(
        go.Scatter(
            x=buy_signals.index,
            y=buy_signals["low"],
            mode="markers",
            name="Buy Signal",
            marker=dict(symbol="triangle-up", size=10, color="green", line=dict(width=1, color="darkgreen")),
        ),
        row=1,
        col=1,
    )

    # Add sell signals (red triangles)
    sell_signals = candles_with_mrc[candles_with_mrc["signal"] == -1]
    fig.add_trace(
        go.Scatter(
            x=sell_signals.index,
            y=sell_signals["high"],
            mode="markers",
            name="Sell Signal",
            marker=dict(symbol="triangle-down", size=10, color="red", line=dict(width=1, color="darkred")),
        ),
        row=1,
        col=1,
    )

    # Add the condition to the second row
    fig.add_trace(
        go.Scatter(
            x=candles_with_mrc.index,
            y=candles_with_mrc["condition"],
            mode="lines",
            name="Condition",
            line=dict(color="#00FFFF", width=2),
        ),
        row=2,
        col=1,
    )

    # Update layout for dark theme and technical colors
    fig.update_layout(
        height=800,
        width=1800,
        title_text="Mean Reversion Channel with Multiple Timeframe Analysis",
        xaxis_rangeslider_visible=False,
        template="plotly_dark",
        hovermode="x unified",
        hoverdistance=100,
        spikedistance=1000,
    )

    # Update y-axis labels and colors
    fig.update_yaxes(title_text="Price", row=1, col=1, color="#FFFFFF")
    fig.update_yaxes(title_text="Condition", row=2, col=1, color="#00FFFF")

    # Update x-axis color and add spikes
    fig.update_xaxes(color="#FFFFFF", showspikes=True, spikecolor="white", spikesnap="cursor", spikemode="across")

    return fig


# Usage example:
config = MeanReversionChannelConfig(length=200)
candles.add_feature(MeanReversionChannel(config))

# Assuming you have implemented the MTF analysis in your MeanReversionChannel class
df = candles.data
df = compute_signal(df)

fig = plot_mean_reversion_channel(candles, df)
fig.show()